In [1]:
#download data links:
#https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip


## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, 
you can import the model <code>VGG16</code> from <code>keras.applications</code>.
You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

In [2]:
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input

In [3]:
num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100

In [4]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

In [5]:
train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.


In [6]:
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')

Found 9501 images belonging to 2 classes.


In [7]:
model = Sequential()
model.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

In [8]:
model.add(Dense(num_classes, activation='softmax'))

In [9]:
model.layers

In [10]:
model.layers[0].trainable = False

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 dense (Dense)               (None, 2)                 1026      
                                                                 
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [12]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

In [14]:
fit_history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Epoch 1/2
301/301 [==============================] - 158s 492ms/step - loss: 0.0965 - accuracy: 0.9679 - val_loss: 0.0270 - val_accuracy: 0.9926
Epoch 2/2
301/301 [==============================] - 143s 474ms/step - loss: 0.0207 - accuracy: 0.9951 - val_loss: 0.0156 - val_accuracy: 0.9956


In [15]:
model.save('classifier_vgg_model.h5')

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.

In [17]:
model_resnet = keras.models.load_model('classifier_resnet_model.h5')

In [18]:
print(model_resnet.summary())
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense (Dense)               (None, 2)                 4098      
                                                                 
Total params: 23,591,810
Trainable params: 4,098
Non-trainable params: 23,587,712
_________________________________________________________________
None
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 dense (Dense)               (None, 2)                 1026      
                                                                 
Total params: 1

In [19]:
evaluate_generator = data_generator.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical',shuffle=False)

Found 500 images belonging to 2 classes.


In [20]:
loss1, accuracy1 = model.evaluate(evaluate_generator)
print('Model VGG16: Loss:', loss1, 'Accuracy :', accuracy1)

5/5 [==============================] - 2s 354ms/step - loss: 0.0129 - accuracy: 0.9960
Model VGG16: Loss: 0.012870730832219124 Accuracy : 0.9959999918937683


In [21]:
loss2, accuracy2 = model_resnet.evaluate(evaluate_generator)
print('Model Resnet50: Loss:', loss2, 'Accuracy:', accuracy2)

5/5 [==============================] - 5s 299ms/step - loss: 0.0024 - accuracy: 1.0000
Model Resnet50: Loss: 0.002413706621155143 Accuracy: 1.0


## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

In [23]:
predict_generator = data_generator.flow_from_directory(
    'concrete_data_week4/test',
    batch_size=5,
    class_mode='categorical',
    seed=24
    )

Found 500 images belonging to 2 classes.


In [24]:
images,labels=predict_generator.next()
predictions = model.predict(images, steps=5)

5/5 [==============================] - 2s 7ms/step


In [25]:
import numpy as np
import matplotlib.pyplot as plt

classes=["Negative","Positive"]
for i in range(5):
    print(f"Real {i}. sample label :",classes[np.argmax(labels[i])])
    print(f"Pred {i}. sample label :",classes[np.argmax(predictions[i])])

Real 0. sample label : Positive
Pred 0. sample label : Positive
Real 1. sample label : Negative
Pred 1. sample label : Negative
Real 2. sample label : Negative
Pred 2. sample label : Negative
Real 3. sample label : Negative
Pred 3. sample label : Negative
Real 4. sample label : Positive
Pred 4. sample label : Positive
